In [1]:
import os
from dotenv import load_dotenv
import dspy
import logging

logging.getLogger().setLevel(logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# set your api key (if needed)
load_dotenv("../../.env")
APIKEY = os.getenv("APIKEY")

# set your model (litellm model strings)
student_model = "openrouter/meta-llama/llama-3.2-3b-instruct"
# student_model = "openrouter/meta-llama/llama-3-8b-instruct"
teacher_model = "openrouter/deepseek/deepseek-chat"
lm = dspy.LM(student_model, api_key=APIKEY, cache=True)
teacher_lm = dspy.LM(teacher_model, api_key=APIKEY, cache=True)
dspy.configure(lm=lm)

In [2]:
from news_app import NewsAppSignature

In [3]:
import glob
import json
from datetime import datetime

data = []
for file in glob.glob("training_data/accepted/*.json"):
    with open(file, "r") as fh:
        tmp = json.load(fh)

        # convert to date
        tmp["publication_date"] = datetime.strptime(tmp["publication_date"], "%Y-%m-%d").date() if tmp["publication_date"] else None

        # remove reasoning from example
        if "reasoning" in tmp:
            del tmp["reasoning"]

        e = dspy.Example(tmp).with_inputs("article_text")
        data.append(e)

print(f"# Examples: {len(data)}")

# Examples: 100


In [5]:
import sys
sys.path.append("../scorer")
from scorer import WordLlamaScorer
from dspy.evaluate import Evaluate
from dspy.teleprompt import MIPROv2


scorer = WordLlamaScorer.from_signature(NewsAppSignature, skip_fields=["article_text", "reasoning"])


teleprompter = MIPROv2(
    metric=scorer,
    auto="medium",
    teacher_settings=dict(lm=teacher_lm),
    num_threads=2
)

catalog = dspy.ChainOfThought(NewsAppSignature)
optimized_program = teleprompter.compile(
    student=catalog.deepcopy(),
    teacher=catalog.deepcopy(),
    trainset=data,
    max_bootstrapped_demos=2,
    max_labeled_demos=2,
    requires_permission_to_run=False,
)

2025/01/11 10:46:01 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 19
valset size: 80

2025/01/11 10:46:01 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/11 10:46:01 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/11 10:46:01 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 10%|█████████████                                                                                                                      | 2/20 [00:00<00:00, 44.96it/s]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/19


 10%|█████████████                                                                                                                      | 2/20 [00:00<00:00, 46.16it/s]


...

2025/01/11 11:20:29 ERROR dspy.utils.parallelizer: Error processing item Example({'article_text': 'Health|The Delta variant caused a spike in deaths among nursing home residents, a study finds.https://www.nytimes.com/2021/10/03/health/delta-deaths-nursing-homes.htmlNursing homes reported nearly 1,800 deaths among their residents and staff in August, the highest monthly toll since February.Credit...Seth Wenig/Associated PressPublished Oct. 3, 2021Updated Oct. 10, 2021Although nursing home deaths from Covid-19 remain dramatically down from their peak at the end of last year, a Kaiser Family Foundation analysis shows a significant uptick in August as the Delta variant swept through the country.After declining for months, largely because of the federal effort to vaccinate residents, the number of deaths rose sharply from July to August. Nursing homes reported nearly 1,800 deaths among their residents and staff in August, which represented the highest monthly toll since February.The finding

Average Metric: 33.88 / 43 (78.8%):  55%|███████████████████████████████████████████████████▋                                          | 44/80 [00:57<01:14,  2.08s/it]

2025/01/11 11:21:06 ERROR dspy.utils.parallelizer: Error processing item Example({'article_text': 'Sports|B.C. Wins Beanpot, Againhttps://www.nytimes.com/2014/02/11/sports/bc-wins-beanpot-again.htmlSports Briefing | College HockeyFeb. 11, 2014Patrick Brown tipped in a shot from the point with 5 minutes 30 seconds minutes to play for the tiebreaking goal and added another in the final minute, lifting Boston College to a school-record fifth straight Beanpot championship with a 4-1 win over Northeastern.Boston College (22-4-3) took over the No. 1 spot in the national rankings.', 'extracted_quotes': [], 'publication_date': datetime.date(2014, 2, 11), 'keywords': ['Boston College', 'Beanpot', 'college hockey', 'Northeastern University', 'national rankings'], 'generated_title': 'B.C. Wins Beanpot, Again', 'primary_category': 'sports', 'content_type': 'reporting', 'mentioned_people': ['Patrick Brown'], 'mentioned_organizations': ['Boston College', 'Northeastern University'], 'mentioned_legisl

Average Metric: 59.20 / 78 (75.9%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [01:33<00:00,  1.17s/it]

2025/01/11 11:21:42 INFO dspy.evaluate.evaluate: Average Metric: 59.20056144868104 / 80 (74.0%)
2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 74.0
2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [61.61, 73.2, 72.11, 74.0]
2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 74.0
2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/11 11:21:42 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 74.0!


In [8]:
from datetime import date

for demo in optimized_program.demos:
    if 'publication_date' in demo and isinstance(demo['publication_date'], date):
        demo['publication_date'] = demo['publication_date'].isoformat()

# Save the state to a JSON file
optimized_program.save("miprov2_llama32_3b.json", save_program=False)
